In [13]:
from officelib.xllib import *
import clipboard, os
def crp():
    clipboard.copy(repr(clipboard.paste()))

In [2]:
xl = Excel()
wb = xl.Workbooks.Add()
ws = wb.Worksheets("Sheet1")
cells = ws.Cells
cr = cells.Range

In [14]:
dropbox = 'C:\\Users\\Nathan\\Documents\\Dropbox\\FRS'
testfp = os.path.join(dropbox, "FRS Tests 181127.xlsx")
twb = xl.Workbooks.Open(testfp)
tws = twb.Worksheets(1)
tcells = tws.Cells
tcr = tcells.Range

In [9]:
for c in xl.Selection:
    v = c.Value
    if v:
        c.IndentLevel = v.count(".")

In [43]:
frs = [x[0] for x in cr("B2:B58").Value2]

In [50]:
test_cells = [(str(int(x[0])),x[9]) for x in tcr("G2:P1462").Value2]

In [53]:
frsi = {i: [] for i in frs}
for tid, items in test_cells:
    if items is not None:
        v = items.splitlines()
        for f in v:
            if f in frsi:
                frsi[f].append(tid)

In [60]:
empty = []
for cell in cr("B2:B58"):
    tests = frsi.get(cell.Value, empty)
    tl = ",".join(tests)
    tgt = cell.Offset(1,4)
    tgt.NumberFormat = "@"
    tgt.Value2 = tl

In [61]:
reqs = [x[0] for x in xl.Selection.Value2]

In [63]:
import importlib
try:
    importlib.reload(jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter)
except NameError:
    pass
import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter
RequirementExtracter = jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter.RequirementExtracter

In [65]:
rex = RequirementExtracter(['URS', 'FRS'])
for c in xl.Selection:
    txt = c.Value2
    refs, ntext = rex._extract_refs(txt.splitlines())
    c.Value2 = ntext
    c2 = c.Offset(1, 2)
    data = "\n".join(a+b for a,b in refs)
    c2.Value2 = data
    

In [66]:
from scripts.tools.issuetracker import IssuetrackerAPI
api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
issues = api.download_issues("pbssoftware")
issues[0]

KeyError: 0

In [93]:
import re
match = re.compile(r".*(URS)([\d\.]+?).*").search
o = set()
for iss in issues.values():
    if iss.sprint_milestone == "Legacy":
        continue
    lines = iss.description.splitlines()
    for line in lines:
        if 'URS3194' in line:
            print(iss.id, iss.subject)
            o.add(iss.id)
            
# import webbrowser
# for iid in o:
#     webbrowser.open_new_tab("https://issue.pbsbiotech.com/issues/%d"%iid)

2922 Power VI
2851 Report Generation
2787 Correct RIO SN reporting
2859 Graph Panel
2998 Verify Action Before Writing to Database


In [83]:
len(issues.values())

648

In [84]:
iss.description

'*+URS4095+* Reactor Name\r\n* *URS4095.1:* Users should be able to set a human-friendly reactor name. \r\n \r\n*+FRS4095+* Reactor name [URS4095]\r\n* *FRS4095.1:* Users will be able to designate a name for the bioreactor. [URS4095.1]\r\n* *FRS4095.2:* The name will be clearly visible in the primary user interface. [URS4095.1]\r\n'

In [85]:
match(iss.description.splitlines()[0])

In [94]:
xl = Excel()

In [97]:
i3194 = '*1 CFR 21 Part 11 Compliance*\r\nNote: Many of the primary requirements needed for CFR 21 Part 11 compliance are intended to be met by a custom shell replacement for explorer.exe ("LabVIEW Shell"). While use of a custom shell is not an explicit CFR 21 Part 11 requirement, it is the most practical solution for us to implement these requirements. High level CFR requirements as well as requirements specific to our implementation are listed here. General Shell requirements are listed in Section 3. \r\n* 1.1 Record Retention and Protection\r\n** 1.1.1 User access must restricted to prevent unauthorized access through software. \r\n*** 1.1.1.1 Prevent modification or manipulation of database\r\n*** 1.1.1.2 Prevent unauthorized addition, deletion, or modification of files through file explorer, dialog, or other means.\r\n*** 1.1.1.3 Prevent unauthorized modification of BIOS settings\r\n*** 1.1.1.4 Prevent required third party programs, particularly Chrome, from bypassing security measures. \r\n** 1.1.2 Generate and store database checksum to guarantee validity of the database\r\n** 1.1.3 Add feature to allow client\'s admin to set up LAN for backup and archiving of databases\r\n** 1.1.4 Correct report generation\r\n** 1.1.5 Create a custom file browsing dialog to prevent unauthorized access\r\n** 1.1.6 Configure our software to work with client email server for reports\r\n** 1.1.7 Add database security file\r\n* 1.2 Audit Trails\r\n** 1.2.1 Record user events in "LabVIEW Shell"\r\n** 1.2.2 Prevent session hijacking in Web UI\r\n** 1.2.3 Correct time recorded to database to be true time of events and process data points\r\n*** We have decided the RIO time should be in UTC.  When an experiment is taking place, and the Atom switches to Daylight Savings in the middle, all data from the RIO recorded to the database before and after the switch must be recorded in the correct local time.  Since the Atom will always know that the RIO is reporting in UTC, it will take every timestamp from the RIO, check the current local time and timezone, and convert that timestamp to local time.\r\n**** Rather than re-calculating and readjusting the conversion formula for each data point, it could re-calculate it at the top of every hour, as that\'s when Daylight Savings transitions happen.  It should also re-calculate it when the database engine starts up.\r\n*** The RIO time might drift, although it would be slow.  To avoid this causing a problem, re-sync the RIO time to the Atom on a regular basis.\r\n*** The times reported to the Web UI must be in the same format they currently are in 2.0\r\n*** This must be able to handle the Atom switching to/from Daylight Savings time\r\n** 1.2.4 The user events for all actions should be clear and accurate\r\n** 1.2.5 User events for all actions should occur in the following sequence:\r\n### Perform the user-requested action\r\n### Wait until the user-requested action has been confirmed through confirmation string\r\n### Write the record\r\n** 1.2.6 Database file history, including archival dates and md5 checksum, must be maintained by the system and unmodifiable by users. \r\n* 1.3 Encryption for Open System\r\n** 1.3.1 Configure the web server for https encryption\r\n*** When users on the bioreactor navigate to the Web UI, the HD should be configured so the page loads without user input\r\n*** Remote users must be able to use the Web UI by navigating to it with Chrome on their own computers\r\n*** When remote users navigate to the Web UI, it would be acceptable if users have to tell Chrome to trust the page each time they navigate to it, and if Chrome indicates that it didn\'t fully trust the site.\r\n** Each bioreactor configured will have its own unique SSL certificate generated during manufacturing as a self-signed root certificate registered to "localhost". \r\n* 1.4 Password Check and Aging\r\n** 1.4.1 Improve password complexity to allow alphanumeric\r\n*** Note: Web UI will still require numeric-only passwords\r\n** 1.4.2 Allow admin to reset password\r\n** 1.4.3 Failed login lock\r\n** 1.4.4 Record failed login to database\r\n** 1.4.5 Password expiration (CFR 21 Part 11.300(b)) ensuring that identification code and password issuance is periodically checked, recalled, or revised (e.g., to cover such events as password aging).\r\n** 1.4.6 Requirements for CFR 21 Part 11.300(d)\r\n*** Use of transaction safeguards to prevent unauthorized use of passwords and/or identification codes, and to detect and report in an immediate and urgent manner any attempts at their unauthorized use to the system security unit, and, as appropriate, to organizational management.\r\n*** Lock out users without Account Management permissions after 10 unsuccessful login attempts, and notify.  Users with Account Management permissions shouldn\'t have Web login permissions (via procedure).  All users will not be allowed another login attempt for 3 seconds in Shell and Web UI.\r\n* 1.5 Control and Reporting of Unauthorized Usage\r\n** 1.5.1 Record User ID for logins and logouts\r\n\r\n*2 Suitability for Manufacturing*\r\n* 2.1 Control Loops\r\n** 2.1.1 Correct PID setup and re-tune PID loops to achieve desired response characteristic\r\n*** User and functional requirements for specific model sizes and controllers will be handled in separate documents\r\n*** Multiple sets of PID parameters may be generated for a controller, to cover for multiple use scenarios such as headspace vs. microsparger DO control. \r\n** 2.1.2 Set up software to expect correct RTD type\r\n** 2.1.3 Improve automatic sensor selection behavior\r\n### For each sensor pair (pH DO and Temp), introduce 3 configuration settings via globals:\r\n#### One, set at the factory, to indicate available hardware configuration ("Present")\r\n#### One, set by users, to indicate which sensors may be used ("Enabled")\r\n#### One, set by users, to indicate which sensor they would prefer to be used ("Preferred")\r\n#### For each setting, indicate sensor A, sensor B, or sensors A and B. \r\n### Selecting Sensors:\r\n#### Only trigger alarms for sensors that are actually Present and Enabled. \r\n#### If the machine only has hardware to support 1 sensor, use that sensor regardless of other settings. \r\n#### Never use a sensor the user has Disabled, unless that is the only sensor Present. \r\n#### Always use the user Preferred sensor, unless that sensor is not Present or in range. \r\n#### If both sensors are Present and Enabled, automatically switch sensors if the In Use sensor goes out of range. \r\n#### Refer to attachment "Sensor_Changes_PV_160208_1.png" for further details. \r\n* 2.2 Alarm Feature Improvements\r\n** 2.2.1 Remove outdated and unused alarms\r\n** 2.2.2 Add audible alarm for unintentional shutdown of bioreactor\r\n** 2.2.3 Add audible alarms for blown fuses\r\n** 2.2.4 Allow acknowledgement of alarms to span database archive events\r\n* 2.3 Calibration\r\n** 2.3.1 Prevent calibrations that can result in "Inf" value for m and b\r\n** 2.3.2 Add pressure \'Zero\' calibration button\r\n** -2.3.3 Load m and b correctly in calibration menu under all test cases-\r\n** 2.3.4 Get raw values for Web UI pH and DO calibration\r\n** 2.3.5 Expose calibration options for MFC configuration, including slope and intercept correction.\r\n* 2.4 Recipes\r\n** 2.4.1 Correct implementation of recipe looping\r\n** 2.4.2 Allow recipes to edit all relevant data types\r\n** 2.4.3 Eliminate "Unknown Command" in recipe steps report\r\n* 2.5 Corrections and improvments to internal machinery:\r\n2.5.1 Eliminate web server freezing and crashing under specific conditions\r\n** 2.5.2 Eliminate getDORAValues error when RIO is disconnected\r\n** 2.5.3 Report RIO SN correctly\r\n* 2.6 Ease of use and ease of validation\r\n** 2.6.1 Add granular user permissions to all categories and subcategories of access\r\n*** 2.6.1.1 Allow administrative access for administrators and lab directors\r\n*** 2.6.1.2 Allow restriction to operation or observation only for lab technicians\r\n** 2.6.2 Use SysPrep so that our manufacturing process can be supported by Microsoft\r\n** 2.6.3 Encrypt email passwords\r\n** 2.6.4 Remove unnecessary System Variables, globals, typedefs, and sub VIs\r\n** 2.6.5 Improvements to report generation\r\n*** Users must be able to generate reports from the LabVIEW Shell for the active and archived databases\r\n*** When selecting from a list of batches, users must be able to tell that a particular batch is either continued in the next database, or is continued from the previous database\r\n*** The generated report will have headers to help users identify the batch or time it is associated with\r\n* 2.7 Securing PBS software against piracy\r\n** 2.7.1 The auth sb RIO should have a secure, random password\r\n\r\n*3 LabVIEW Shell Misc Requirements*\r\nNote: The primary purpose of creating the LabVIEW Shell is to satisfy requirements of CFR 21 part 11, covered in URS Section 1. This section covers general requirements to create a suitable interface for customer use and any other requirements specific to the Shell UI\'s design which are required to satisfy CFR 21 part 11. \r\n* 3.1 Power functions - allow user to perform the following functions in Web Server and LabVIEW Shell:\r\n** Shut down the Windows computer\r\n** Restart the Windows computer\r\n** Log out the Windows user\r\n* 3.2 Shell is to utilize a custom error dialog, which is to be consistent across all parts of the program\r\n* 3.3 Shell is to have a graph panel, which is to update graphs more frequently than in the Web UI.  Also, show graphs for the following parameters, which are not graphed in the Web UI:\r\n** Filter Oven\r\n** Pressure\r\n** Level\r\n** Gas Voltages\r\n** Main Gas flow (for Mag drive)\r\n* 3.4 All custom interfaces provided by the LabVIEW Shell must be accessible and usable without use of an external keyboard. \r\n* 3.5 Unless otherwise specified, all functionality available in the Hello 2.0 Service UI should be available to users through the LabVIEW shell. \r\n* 3.6 Users must be automatically logged out due to inactivity after 10 minutes. Inactivity is defined as no mouse movement, mouse clicks, or keyboard inputs. \r\n** 3.6.1 A small popup should warn users that the logout timer is about to expire, after 9 minutes of inactivity, and disappear when the timer expires or is reset. '

In [111]:
match = re.compile(r"^([\*\s]*)?([\d\.]+)\s*(.*)").match
lines = i3194.splitlines()
out = []
i = 0
it = []
while i < len(lines):
    l = lines[i]
    m = match(l)
    if m:
        if it:
            t = it[0], "\n".join(it[1:])
            out.append(t)
        it = [m.group(2), m.group(3)]
    else:
        it.append(l)
    i += 1
out.append((it[0], "\n".join(it[1:])))
out

[('1',
  'CFR 21 Part 11 Compliance*\nNote: Many of the primary requirements needed for CFR 21 Part 11 compliance are intended to be met by a custom shell replacement for explorer.exe ("LabVIEW Shell"). While use of a custom shell is not an explicit CFR 21 Part 11 requirement, it is the most practical solution for us to implement these requirements. High level CFR requirements as well as requirements specific to our implementation are listed here. General Shell requirements are listed in Section 3. '),
 ('1.1', 'Record Retention and Protection'),
 ('1.1.1',
  'User access must restricted to prevent unauthorized access through software. '),
 ('1.1.1.1', 'Prevent modification or manipulation of database'),
 ('1.1.1.2',
  'Prevent unauthorized addition, deletion, or modification of files through file explorer, dialog, or other means.'),
 ('1.1.1.3', 'Prevent unauthorized modification of BIOS settings'),
 ('1.1.1.4',
  'Prevent required third party programs, particularly Chrome, from bypas

In [113]:
c = xl.Selection
e = c.Offset(len(out), len(out[0]))
cr = xl.Selection.Parent.Cells.Range
cr(c,e).Value = out